<a href="https://colab.research.google.com/github/mathrhino/pytorch/blob/main/torch_mnist_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch

import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim

from torchvision import datasets, transforms

In [36]:
seed =1 
lr = 0.001
momentum =0.5

batch_size =64
test_batch_size=64

epochs =1 
no_cuda =False
log_interval =100

In [37]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1,20,5,1)
    self.conv2 = nn.Conv2d(20,50,5,1)
    self.fc1= nn.Linear(4*4*50, 500)
    self.fc2 = nn.Linear(500, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x,2,2)
    x= F.relu(self.conv2(x))
    x=F.max_pool2d(x,2,2)
    x=x.view(-1, 4*4*50)
    x=F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)

In [38]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs ={'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
                ])),
            batch_size=batch_size,
            shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                                                                         transforms.ToTensor(),
                                                                         transforms.Normalize((0.1307,),(0.3081,))
    ])),
    batch_size=test_batch_size, shuffle=True, **kwargs
)

In [39]:
model = Net().to(device)
optimizer=optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [41]:
for epoch in range(1, epochs +1):
  model.train()

  for batch_idx, (data, target) in enumerate(train_loader):
    data, target =data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % log_interval ==0:
      print('Train Epoch: {}[{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx *len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()
      ))
  model.eval()
  test_loss=0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
  test_loss /= len(test_loader.dataset)

  print('\tTest set: Average loss: {:4f}, Accuracy: {}/{} ({:.0f}%)\n'. format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)
  ))

Train Epoch: 1[0/60000 (0%)]	Loss: 2.295366
Train Epoch: 1[6400/60000 (11%)]	Loss: 2.220284
Train Epoch: 1[12800/60000 (21%)]	Loss: 2.136019
Train Epoch: 1[19200/60000 (32%)]	Loss: 1.953842
Train Epoch: 1[25600/60000 (43%)]	Loss: 1.498457
Train Epoch: 1[32000/60000 (53%)]	Loss: 1.254111
Train Epoch: 1[38400/60000 (64%)]	Loss: 0.842578
Train Epoch: 1[44800/60000 (75%)]	Loss: 0.706257
Train Epoch: 1[51200/60000 (85%)]	Loss: 0.586765
Train Epoch: 1[57600/60000 (96%)]	Loss: 0.612415
	Test set: Average loss: 0.486875, Accuracy: 8609/10000 (86%)

